# C13 example: How many times has each K2 pixel been telemetered?

C13 example

In this demo notebook we are going to identify which pixels were observed in long cadence (LC) during campaign 13.  Our final goal is to have a boolean mask the same dimensions as an FFI, with either 1 (was observed) or 0 (not observed).  Our approach will be simply to read in every single TPF and access the `APERTURE` keyword.  Values of `APERTURE` greater than zero are observed, while zero values were not.  We will also need to read the FITS header to determine the *x,y* coordinate of the corner of the TPF.  Finally we will programmatically fill in a count array with ones or zeros.

Make the Jupyter Notebook fill the screen.

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.utils.console import ProgressBar
import logging
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Open the Campaign 13 FFI to mimic the dimensions and metadata.

In [3]:
hdu_ffi = fits.open('/Volumes/Truro/ffi/ktwo2017079075530-c13_ffi-cal.fits')

Open the [k2 Target Index](https://github.com/barentsen/k2-target-index) csv file, which is only updated to Campaign 13 at the time of writing.

In [4]:
df = pd.read_csv('../../k2-target-index/k2-target-pixel-files.csv.gz')

For this notebook, we will just focus on Campaign 13.  We can generalize our result in the future by using *all* the campaigns.

In [5]:
df = df[df.campaign == 13].reset_index(drop=True)

In [6]:
hdu_ffi[10].name

'MOD.OUT 4.2'

The Kepler FFI CCDs are referred to by their **"mod.out" name**, rather than by "channel".  The nomenclature is a relic of how the CCD readout electronics were configured.  
What matters here is that:  
1. We will have to mimic the formatting to programmatically index into the FFI counts.
2. Some modules have more target pixel files on them than others, by chance or by astrophysical design.

In [7]:
df.groupby(['module', 'output']).filename.count().head(10)

module  output
2       1         342
        2         347
        3         404
        4         416
6       1         345
        2         983
        3         598
        4         283
8       1         573
        2         929
Name: filename, dtype: int64

In [8]:
df['mod_str'] = "MOD.OUT "+df.module.astype(str)+'.'+df.output.astype(str)

In [9]:
df['mod_str'].tail()

26282    MOD.OUT 17.1
26283    MOD.OUT 16.4
26284    MOD.OUT 13.4
26285     MOD.OUT 8.4
26286    MOD.OUT 19.4
Name: mod_str, dtype: object

We'll make an **"FFI Counts" array**, which is an array the same dimensions as the FFI, but with values of whether a pixel was telemetered in Long Cadence or not.

In [10]:
hdu_counts = hdu_ffi

In [11]:
for i, el in enumerate(hdu_ffi[1:]):
    hdu_counts[el.name].data = hdu_counts[el.name].data*0.0

For the next part, you'll need a local hard drive full of all the K2 target pixel files.  It's a single line `wget` script to [MAST](https://archive.stsci.edu/pub/k2/target_pixel_files/c13/).  It's possible that the downloading process would corrupt a few target pixel files, and you wouldn't necessarily know.  So we will also set up a log of the failed attempts to open a target pixel file.

In [12]:
local_dir = '/Volumes/burlingame/TPFs/c13/'

In [13]:
logging.basicConfig(filename='../data/C13_failed_TPFs.log',level=logging.INFO)

In [14]:
mod_list = df.mod_str.unique()
mod_list

array(['MOD.OUT 8.1', 'MOD.OUT 8.4', 'MOD.OUT 23.1', 'MOD.OUT 23.2',
       'MOD.OUT 14.1', 'MOD.OUT 9.1', 'MOD.OUT 8.3', 'MOD.OUT 22.2',
       'MOD.OUT 24.1', 'MOD.OUT 6.1', 'MOD.OUT 19.3', 'MOD.OUT 15.1',
       'MOD.OUT 13.3', 'MOD.OUT 24.2', 'MOD.OUT 9.4', 'MOD.OUT 22.1',
       'MOD.OUT 22.4', 'MOD.OUT 23.3', 'MOD.OUT 14.2', 'MOD.OUT 22.3',
       'MOD.OUT 11.3', 'MOD.OUT 6.4', 'MOD.OUT 6.3', 'MOD.OUT 12.1',
       'MOD.OUT 12.3', 'MOD.OUT 24.4', 'MOD.OUT 9.2', 'MOD.OUT 11.4',
       'MOD.OUT 2.3', 'MOD.OUT 10.3', 'MOD.OUT 23.4', 'MOD.OUT 2.4',
       'MOD.OUT 19.1', 'MOD.OUT 24.3', 'MOD.OUT 16.4', 'MOD.OUT 16.3',
       'MOD.OUT 16.1', 'MOD.OUT 16.2', 'MOD.OUT 11.1', 'MOD.OUT 11.2',
       'MOD.OUT 17.4', 'MOD.OUT 17.3', 'MOD.OUT 6.2', 'MOD.OUT 12.4',
       'MOD.OUT 17.1', 'MOD.OUT 17.2', 'MOD.OUT 12.2', 'MOD.OUT 18.3',
       'MOD.OUT 2.1', 'MOD.OUT 18.2', 'MOD.OUT 13.4', 'MOD.OUT 18.4',
       'MOD.OUT 2.2', 'MOD.OUT 18.1', 'MOD.OUT 13.1', 'MOD.OUT 13.2',
       'MOD.OUT 19.2

Now set up a big for-loop that:
1. Reads in a TPFs 
2. Aligns its corner in the FFI frame
3. Adds a boolean mask to the **FFI Counts** array
4. Optionally logs any problem TPFs for spot-checking later
5. Incrementally saves the FFI Counts array to a FITS file

In [ ]:
for mod_out, group in df.groupby('mod_str'):
    print(mod_out, end='  ')
    mod_out_tpfs = group.url.str[59:].values
    with ProgressBar(len(mod_out_tpfs), ipython_widget=True) as bar:
        for i, tpf_path in enumerate(mod_out_tpfs):
            bar.update()
            try:
                hdu_tpf = fits.open(local_dir+tpf_path)
                hdr = hdu_tpf['TARGETTABLES'].header
                cx, cy = hdr['1CRV4P'], hdr['2CRV4P']
                sx, sy = hdu_tpf['APERTURE'].shape
                counts = hdu_tpf['APERTURE'].data > 0
                hdu_counts[mod_out].data[cy:cy+sy, cx:cx+sx]=counts.T # Double check this!
            except:
                logging.info(tpf_path+" had a problem: cx:{}, sx:{}, cy:{}, sy:{}".format(cx, sx, cy, sy))
            hdu_tpf.close()
    hdu_counts.writeto('../data/FFI_counts/C13_FFI_mask.fits', overwrite=True)

MOD.OUT 10.1  

A Jupyter Widget


MOD.OUT 10.2  

A Jupyter Widget


MOD.OUT 10.3  

...

A Jupyter Widget


MOD.OUT 11.2  

A Jupyter Widget


MOD.OUT 11.3  

A Jupyter Widget

Ok, it took 8 minutes for 3 channels, or about 160 seconds per channel.  There are about 72 channels per pointing, so:

In [ ]:
160.0*72/60.0

In [ ]:
192.0/60

Counting all the pixels will take about 3.2 hours.  That's a long time!  Meep!  
Let's have the for loop incrementally save each channel count so we can start working with the count data.

In [ ]:
plt.figure(figsize=(14,14))
plt.imshow(hdu_counts['MOD.OUT 8.1'].data, interpolation='none', );

The end!